In [2]:
# 二级模型单条测试
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import tensorflow as tf
from hanlp.components.pos import RNNPartOfSpeechTagger

tagger1 = RNNPartOfSpeechTagger()
save_moedl='../model/pos1030first/'
tagger1.load(save_moedl)
tagger2 = RNNPartOfSpeechTagger()
save_moedl='../model/pos1030second/'
tagger2.load(save_moedl)

In [3]:
def Process(addr, tag):
    strList = []
    tagCur = ''
    for i in range(len(tag)):
        if tag[i] != 'X' and tag[i] != 'M':
            if i != 0:
                strList.append(',')
                strList.append(tagCur)
                strList.append(',')
            strList.append(addr[i])
            tagCur = tag[i]
        else:
            strList.append(addr[i])
    strList.append(',')
    strList.append(tagCur)
    return ''.join(strList)

In [6]:
def predict(addStr):
    addList = [list(addStr)]
    init = tagger1.predict(addList)[0]
    strNextList = []
    start = -1
    for i in range(len(addList[0])):
        if init[i] == 'RX':
            start = i
            break
    if start == -1:
        return ' '.join(init[:])
    else:
        second = tagger2.predict([addList[0][start:]])[0]
        return ' '.join(init[:start]) + ' ' + ' '.join(second)
    
def nlpWordsTest(rdAddress):
    address=list(rdAddress)
    M1 = tagger1.predict(address)
    address1 = Process(address,M1)
    M2 = predict(address)
    M2 = M2.split()
    address2 = Process(address,M2)
    print(address1)
    print(address2+'\n')

In [29]:
nlpWordsTest('安徽省六安市霍邱县城关地税分局')

安徽省,R1,六安市,R2,霍邱县,R3,城关地税分局,RX
安徽省,R1,六安市,R2,霍邱县,R3,城关地税分局,R24

